<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/%E5%8F%8D%E5%AF%BE%E6%84%8F%E8%A6%8B%E6%8A%BD%E5%87%BABERT/Extract_opposite_with_BERT_negapoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPU使用制限　残り時間
!cat /proc/uptime | awk '{printf("残り時間 : %.2f", 12-$1/60/60)}'

残り時間 : 11.99

In [ ]:
'''
反対意見を取り出すプログラムパート２

入力：文章（全角140字以内)　＋　検索対象のcsvファイル
出力： csvファイル中の文章の中で、最も意見が入力と違うもの


処理手順
- csvファイル（検索対象）のベクトル化・ネガポジ分析を終わらせる
- 入力文章のネガポジ分析 （スコアを出す）
- 入力文章のスコアSとすると、-Sに最も近い値を持つ文章を出力

'''

'\n反対意見を取り出すプログラムパート２\n\n入力：文章（全角140字以内)\u3000＋\u3000検索対象のcsvファイル\n出力： csvファイル中の文章の中で、最も意見が入力と違うもの\n\n\n処理手順\n- csvファイル（検索対象）のベクトル化・ネガポジ分析を終わらせる\n- 入力文章のネガポジ分析 （スコアを出す）\n- 入力文章のスコアSとすると、-Sに最も近い値を持つ文章を出力\n\n'

In [ ]:
# Google Driveに接続
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # 作業ディレクトリへ移動
# !pwd
# # %cd ../../
# !ls
# %cd drive/MyDrive/学業関連_from_OneDrive/卒論/制作物ソースコード/反対意見抽出BERT

/content
drive		      lightning_logs  model_transformers  text
ldcc-20140209.tar.gz  model	      sample_data
/content/drive/MyDrive/学業関連_from_OneDrive/卒論/制作物ソースコード/反対意見抽出BERT


In [1]:
# データセット：chABSA-dataset
# ここではネガティブ、ポジティブ、ニュートラル（中立）の３カテゴリに、単語ごとの分類を行っている

# データダウンロード
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
!unzip chABSA-dataset.zip

--2022-10-03 01:32:06--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.195.84
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.195.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip’

chABSA-dataset.zip  100%[===================>] 705.84K   623KB/s    in 1.1s    

2022-10-03 01:32:08 (623 KB/s) - ‘chABSA-dataset.zip’ saved [722777/722777]

Archive:  chABSA-dataset.zip
   creating: chABSA-dataset/
  inflating: chABSA-dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/chABSA-dataset/
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflating:

In [2]:
# 各種ライブラリインポート
# いつものライブラリに加えて、ファインチューニングと性能評価を行うPyTorch Lightningも使用
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# またNo module named ‘torchtext.legacy’エラーが出たため
# https://masaki-note.com/2022/05/27/torchtext_legacy_error/
# に従い以下を実行
# !pip install --upgrade torchtext==0.9.1

# いろいろなライブラリたち
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 14.5 MB/s 
     |████████████████████████████████| 486 kB 67.9 MB/s 
     |████████████████████████████████| 13.4 MB 59.0 MB/s 
     |████████████████████████████████| 582 kB 50.5 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
     |████████████████████████████████| 163 kB 72.5 MB/s 
     |████████████████████████████████| 880 kB 75.3 MB/s 
     |████████████████████████████████| 419 kB 70.1 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=be77d539a9f0ca6ce159e5bde908af1f6feb5b2f8505d7e755baf0c5cd4e5ba9
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=803c2247202af67500d6908b903934e24d5bff495f7b147ebb74c1d345b98d9f
  Stored in dire

In [3]:
# 手順1 : まずはトークナイザを定義

# 日本語事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)



Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [4]:
# マルチラベル分類用BERTも定義
# 実装
# torch.nn　の nn は多分 neural network
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html

class BertForSequenceClassificationMultiLabel(torch.nn.Module):

    def __init__(self, model_name, num_labels):
        super().__init__()

        # BertModelをロード
        self.bert = BertModel.from_pretrained(MODEL_NAME)

        # 線形変換の初期化
        # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        # https://free.kikagaku.ai/tutorial/basic_of_deep_learning/learn/pytorch_beginners
        # 線形変換：w1x1 + w2x2 + ......  みたいな。重みは学習する
        self.linear = torch.nn.Linear (
            self.bert.config.hidden_size, num_labels
        )

  
    # forward : データを入力し、BERTの最終層の出力を得る
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        # データ(input_ids, attention_mask([PAD]の位置を表す), token_type_ids(何文目かを表す))をBERTにを入力
        # attention_maskって、[PAD]に対応するところは0でそれ以外は1のベクトル
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # bert_outputのうち、最終層の出力だけ取得
        last_hidden_state = bert_output.last_hidden_state

        # [PAD]以外のトークンで隠れ状態の平均を取る
        # \ (macだとoption + ¥) で改行できるっぽい
        # unsqueeze(num) : torch.Tensorの次元をnumだけ上げてくれる
        # https://pytorch.org/docs/stable/generated/torch.unsqueeze.html
        # sum : https://zanote.net/python/pytorch-torch-sum/
        averaged_hidden_state = \
            (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True)

        #　上記をさらに線形変換
        scores = self.linear(averaged_hidden_state)

        # 出力形式をいつものBertModelと同様になるように整える
        output = {'logits' : scores}

        # labelsが入力に含まれていたら、損失を計算して出力する
        # BCE : バイナリクロスエントロピー
        # torch.nn.BCEWithLogitsLoss : シグモイド関数で変換した値のバイナリクロスエントロピを計算する関数
        # https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
        # https://runebook.dev/ja/docs/pytorch/generated/torch.nn.bcewithlogitsloss
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss() (scores, labels.float())
            output['loss'] = loss

        # 属性でアクセスできるようにする
        # ちょっとよく分からん・・・
        output = type('bert_output', (object, ), output)

        return output

In [5]:
# 手順2 : Bertのファインチューニングを行うため、pytorch_lightningのクラスを定義
# PyTorch Lightningの定義
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name=model_name, num_labels=num_labels
        )


    def training_step(self, batch, batch_idx):
        # lossを返す
        output = self.bert_scml(
            **batch
        )

        loss = output.loss
        self.log('train_loss', loss)
        return loss

    
    def validation_step(self, batch, batch_idx):
        # lossを求める
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        # accuracyを求める
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)

        scores = output.logits
        labels_predicted = (scores > 0).int()

        num_correct = (labels_predicted == labels).all(-1).sum().item()
        accuracy = num_correct / scores.size(0)

        self.log('accuracy', accuracy)


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [6]:
# 手順3 : 学習データを整形し、データローダを定義

'''
BERTは一度に512個までしかトークンを受け取れない
今回は１度に１２８トークン送り込むようにする

前処理手順１
各データを次のキーをもつ辞書式にしてデータローダに突っ込む
input_ids
attention_mask
token_type_ids
labels
'''



# 最大トークン数 : 半角twitter文字数上限 + [CLS][SEP]
max_length = 280 + 2

# データを読み込んでは上記の形式の辞書に追加
# 上記のデータセットを整形
# 特に、カテゴリをMulti-hot ベクトル[negative, neutral, positive]で表す

category_id = {'negative': 0, 'neutral': 1, 'positive': 2}

# 整形後データセットを入れる箱
dataset = []
import json
for file in glob.glob('chABSA-dataset/*json'):
    # データを開く
    data = json.load(open(file))

    # 各データから文章（text）を抜き出し、ラベル('labels')を作成
    for sentence in data['sentences']:
        # 文章を１文丸ごと取得
        text = sentence['sentence']

        # ラベル初期化
        labels = [0, 0, 0]

        # opinions(単語ごとの分類などが入った属性)を展開
        for opinion in sentence['opinions']:
            # category_idでpolarityに対応する数字を取得し、その位置に1を入れる
            labels[category_id[opinion['polarity']]] = 1
        
        # 整形後データ（１文）
        sample = {'text': text, 'labels': labels}
        # 整形後データセットに追加
        dataset.append(sample)

# 整形後のデータの入れ物
dataset_for_loader = []
# トークン数は最大128まで
max_length = 128

for sample in dataset:
    text = sample['text']
    labels = sample['labels']

    encoding = tokenizer(
        text,
        max_length=max_length,
        padding = 'max_length',
        truncation = True
    )

    encoding['labels'] = labels
    encoding = { k : torch.tensor(v) for k, v in encoding.items() }

    dataset_for_loader.append(encoding)

# 前処理手順2 : データセットを分割
'''
データセットを

学習データ60%
検証データ20%
テストデータ20%

になるようにランダムに分割する

'''

# データセット並べ替え
random.shuffle(dataset_for_loader)

# それぞれのデータ分割のデータ数計算
n = len(dataset_for_loader)
n_train = int(n*0.6)
n_val = int(n*0.2)

# 分割
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:(n_train+n_val)]
dataset_test = dataset_for_loader[(n_train + n_val):]


# データローダを定義
dataloader_for_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
)

dataloader_for_val = DataLoader(
    dataset_val, batch_size=256
)
dataloader_for_test = DataLoader(
    dataset_test, batch_size=256
)

In [7]:
# 手順４　：　ファインチューニング実行
# ファインチューニングの設定を行う

# 学習時にモデルの重みを保持する条件を指定
# つまり、いろいろパラメータをいじっていく中で、どんなパラメータであれば保存するのか
checkpoint = pl.callbacks.ModelCheckpoint(
    # val_lossを監視する
    monitor='val_loss',

    # val_lossが最小のモデルを保存
    mode = 'min',

    # val_lossが最も小さいベストモデル１件を保存
    save_top_k=1,

    # モデルの重みのみを保存する
    save_weights_only=True,

    # モデルファイルを保存するディレクトリ
    dirpath='model/'

)


# 学習方法を指定
trainer = pl.Trainer(
    # 学習にGPUを１つ使う
    gpus=1,

    # 学習を10エポック行う
    # max_epochs=10,
    # めっちゃ時間かかるので3にしよう
    max_epochs=3,

    # どんな時にモデルの重みを保存するのか指定
    callbacks = [checkpoint]
)

# さて、ファインチューニングを実際に行え
model = BertForSequenceClassificationMultiLabel_pl(
    model_name=MODEL_NAME,
    num_labels=3,
    lr=1e-5
)

trainer.fit(model, dataloader_for_train, dataloader_for_val)
test = trainer.test(dataloaders=dataloader_for_test)
print(test[0])
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

# 学習完了！
# ベストモデルを保存
best_model_path = checkpoint.best_model_path
print('ベストモデルのファイル：', best_model_path)
print('ベストモデルの検証データに対する損失', checkpoint.best_model_score)

# 学習時の学習データや検証データに対する損失の時間変化を見るには以下
%load_ext tensorboard
%tensorboard --logdir

# 検証データがU字型 -> オーバーフィット気味

# ファインチューニングで得たモデルをテストデータで評価
test = trainer.test(dataloaders=dataloader_for_test)
print(f'Accuracy: {test[0]["accuracy"]: .2f}')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summ

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.8938775658607483
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'accuracy': 0.8938775658607483}
Accuracy: 0.89
ベストモデルのファイル： /content/model/epoch=1-step=230.ckpt
ベストモデルの検証データに対する損失 tensor(0.1195, device='cuda:0')


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1445: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/model/epoch=1-step=230.ckpt
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/model/epoch=1-step=230.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.8938775658607483
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy:  0.89


In [ ]:
# Transformers対応のモデルを./model_transformesに保存
# これは直接読み込める形で保存される
# model.bert_scml.save_pretrained('./model_transformers_bert_negapoji')

In [ ]:
# # 学習結果を直接読み込むとは
# # 以下のように、いつもモデル名を書いていたところにパスを埋め込むことで達成
# # 2回目以降はこれを使えば良い
# bert_scml = BertForSequenceClassificationMultiLabel.from_pretrained(
#     './model_transformers'
# )

In [8]:
# モデル読み込み
best_model_path=checkpoint.best_model_path
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# 手順5 : 入力されたcsvファイルをbert_scmlに渡して、分類スコアを返してもらう


# PyTorch Lightningモデルのロード
# model = BertForSequenceClassification_pl.load_from_checkpoint(
#     best_model_path
# ) 

# # Transformers対応のモデルを./model_transformesに保存
# # これは直接読み込める形で保存される
# model.bert_scml.save_pretrained('./model_transformers')


# 入力されたcsvファイルを読み込む
# csvファイル読み込み方：https://qiita.com/motoki1990/items/0274d8bcf1a97fe4a869
import pandas as pd
csv_file_name = "KDDI 2022-07-02~2022-07-02 relevancy.csv"
csv_input = pd.read_csv(filepath_or_buffer=csv_file_name,  sep=",")

# 辞書形式に直しましょう
csv_input = csv_input.to_dict()

# ツイートを１つずつ取り出してはスコアを出す
# 出したスコアはscoresに入力
scores = []
tweet_list = []

# 時間かかるので、今回はcsvファイルの最初の10%(100ツイート程度)を使用
for tweet_num in tqdm(range(int(len(csv_input["ツイート内容"]) // 10.0))):
    # csv_input["ツイート内容"] = { tweet番号: '本文', tweet番号: '本文', ...... }
    tweet = csv_input['ツイート内容'][tweet_num]
    tweet_list.append(tweet)

# とりあえず符号化
max_length = 280+2
encoding = tokenizer(
    tweet_list,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

encoding = {k:v.cuda() for k,v in encoding.items()}
    

with torch.no_grad():
    output = bert_scml(**encoding)

    scores = output.logits

100%|██████████| 100/100 [00:00<00:00, 567564.82it/s]


In [28]:
# 手順6 : 入力文章（１文）をbert_scに渡して、分類スコアを出してもらう
input_sentence = 'お！圏外ではなくなった！KDDIはよ頑張れ*(°̀ᗝ°́)و https://t.co/KePRBGyk1U'
input_encoding = tokenizer(
    input_sentence,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

input_encoding = {k:v.cuda() for k, v in input_encoding.items()}
with torch.no_grad():
    output = bert_scml(**input_encoding)
    input_score = output.logits


In [29]:

# 手順7 : 返してもらった分類スコアをもとに、出力する文章を決定する
'''
選定基準

1文に対応するベクトルは
[negative, neutral, positive]

入力文書のスコアをS = - negative + positive として
- S に最も近い値を持つ文章を出力
'''
# print(input_score)
# print(scores)

input_s = - input_score[0][0] + input_score[0][2]
# 検索対象文章のスコアの行列
ss = []

for score in scores:
    ss.append(- score[0] + score[2])

# 元の配列ssでのインデックスと、それに対応するスコアの組の行列
s_sorted = []
for index, ss_factor in enumerate(ss):
    s_sorted.append({'index' : index, 'score' : ss_factor})

# s_sortedをソート
# https://www.mathpython.com/ja/dictionary-sort/
s_sorted = sorted(s_sorted, key=lambda i: i['score'] )

output_tweet = ''
for s in s_sorted:
    if ( - input_s <= s['score']):
        
        output_tweet = csv_input['ツイート内容'][s['index']]
        break

# for文で求まらなかった場合
if len(output_tweet) == 0:
    print('flag')
    output_tweet = csv_input['ツイート内容'][s_sorted[len(s_sorted)-1]['index']]

In [30]:
# 出力
print('入力　：　')
print(input_sentence)

print('最も類似度が低い（反対意見）　：　')
print(output_tweet)

入力　：　
お！圏外ではなくなった！KDDIはよ頑張れ*(°̀ᗝ°́)و https://t.co/KePRBGyk1U
最も類似度が低い（反対意見）　：　
ＫＤＤＩからカタログギフトの優待品、とらふぐ塩辛が到着しました。良きビールのつまみになりそうです✨

＃KDDI
＃KDDI株主優待 https://t.co/4pwRHSTB8x


## 出力記録

```
入力　：　
KDDIの担当者には頑張ってほしい。
最も類似度が低い（反対意見）　：　
KDDI通信障害続く　ヤマト運輸や日本郵便にも影響 
https://t.co/e5tOUz32FT
```

```
入力　：　
KDDIは本当に無能だ。
最も類似度が低い（反対意見）　：　
KDDIより5種のチーズインハンバーグ540gとポークウインナー1kgが届きました。
ボリュームもあり冷凍保存なので保存期間も長くすごく重宝します。 https://t.co/MqAamNf7x6 https://t.co/MZAJ6SMKz6
```

```
入力　：　
KDDIは本当に優秀だ
最も類似度が低い（反対意見）　：　
本日01時35分より当社の通信サービスがご利用しづらい状況が発生しております。
現在、復旧作業に努めておりますが、現時点で判明している原因/影響について情報更新いたしました。
お客さまには多大なご迷惑をおかけしますことを深くお詫び申し上げます。
https://t.co/lnzzxzF8G2
```

```
入力　：　
KDDI見てるか？迷惑かけてるって事を！タクシーの方も🚕困ってたぞ！( *｀ω´)
最も類似度が低い（反対意見）　：　
ＫＤＤＩからカタログギフトの優待品、とらふぐ塩辛が到着しました。良きビールのつまみになりそうです✨

＃KDDI
＃KDDI株主優待 https://t.co/4pwRHSTB8x
```

```
入力　：　
お！圏外ではなくなった！KDDIはよ頑張れ*(°̀ᗝ°́)و https://t.co/KePRBGyk1U
最も類似度が低い（反対意見）　：　
ＫＤＤＩからカタログギフトの優待品、とらふぐ塩辛が到着しました。良きビールのつまみになりそうです✨

＃KDDI
＃KDDI株主優待 https://t.co/4pwRHSTB8x
```

# 上記のテスト（荒いが・・・）の結果を受けて

## よかった点
* 確かに、「感情分析」という点では、割と反対のものを持ってこれている
* これをBERT largeやXLnet を使って実装し直すと、感情分析はいい線いきそう

## よくない点
* 通信障害の話をしている入力に対して、通信障害と関係ないKDDI関連の意見を出力している
* アスペクト分析など必要か？
* しかし、フィルターバブルを避けるという面から言うと、これは「ユーザーがまだ見ていない情報を見せる」に該当する（かもしれない）ので、絶対改善しないといけないかと言われると・・・いや、頑張りますが。
